# 4. Visualization with PCA

This notebook explains how we visualize high-dimensional embeddings in 3D using PCA.

## The Problem: Too Many Dimensions

Our embeddings have **1024 dimensions**. Humans can't visualize 1024-dimensional space!

We need to reduce to **3 dimensions** (x, y, z) for 3D visualization.

## Solution: PCA (Principal Component Analysis)

PCA finds the most important directions in high-dimensional data and projects it down.

Think of it like finding the "best camera angle" to view your data.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from mpl_toolkits.mplot3d import Axes3D

# Create fake high-dimensional data
n_samples = 100
n_features = 1024  # Like our embeddings

# Generate data with some structure
# Class 0: centered at origin
# Class 1: shifted in random direction
# Class 2: shifted in another direction
np.random.seed(42)
class_0 = np.random.randn(30, n_features) * 0.5
class_1 = np.random.randn(35, n_features) * 0.5 + 2
class_2 = np.random.randn(35, n_features) * 0.5 - 2

embeddings = np.vstack([class_0, class_1, class_2])
labels = np.array([0]*30 + [1]*35 + [2]*35)

print(f"Data shape: {embeddings.shape}")
print(f"Labels: {labels[:10]}...")

## Step 1: Standardization

Before PCA, we standardize the features (mean=0, std=1).

**Why?** PCA is sensitive to scale. If one feature has values 0-1000 and another 0-1, PCA will focus on the large-scale feature.

In [ ]:
# Standardize
scaler = StandardScaler()
embeddings_scaled = scaler.fit_transform(embeddings)

print("Before scaling:")
print(f"  Mean: {embeddings.mean():.4f}")
print(f"  Std: {embeddings.std():.4f}")

print("\nAfter scaling:")
print(f"  Mean: {embeddings_scaled.mean():.4f}")
print(f"  Std: {embeddings_scaled.std():.4f}")

## Step 2: Apply PCA

PCA finds the top 3 principal components - the 3 directions that capture the most variance.

In [ ]:
# Reduce to 3D
pca = PCA(n_components=3)
embeddings_3d = pca.fit_transform(embeddings_scaled)

print(f"Reduced shape: {embeddings_3d.shape}")
print(f"\nVariance explained by each component:")
for i, var in enumerate(pca.explained_variance_ratio_):
    print(f"  PC{i+1}: {var:.2%}")
print(f"\nTotal variance explained: {pca.explained_variance_ratio_.sum():.2%}")

**Interpretation:**
- If total variance > 80%, the 3D projection is a good representation
- If total variance < 50%, we're losing a lot of information

For our random data, variance will be low. For real embeddings with structure, it's usually higher.

## Step 3: Visualize in 3D

Now we can plot our 3D coordinates!

In [ ]:
# Create 3D plot
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

# Color by class
colors = ['red', 'blue', 'green']
for class_id in range(3):
    mask = labels == class_id
    ax.scatter(
        embeddings_3d[mask, 0],
        embeddings_3d[mask, 1],
        embeddings_3d[mask, 2],
        c=colors[class_id],
        label=f'Class {class_id}',
        s=50,
        alpha=0.6
    )

ax.set_xlabel('PC1')
ax.set_ylabel('PC2')
ax.set_zlabel('PC3')
ax.set_title('3D PCA Projection of Embeddings')
ax.legend()

plt.tight_layout()
plt.show()

print("If classes cluster together, our embeddings are good!")

## Using PCA with Trained Model Embeddings

In our active learning pipeline, we:
1. Train the model on labeled data
2. Extract intermediate embeddings from the model
3. Apply PCA to reduce to 3D
4. Visualize to see how well the model separates classes

**Important:** We fit PCA once and reuse the transformation!

In [ ]:
# Simulating model embeddings at different training stages
def simulate_model_embeddings(iteration):
    """Simulate embeddings that cluster better over time"""
    noise_level = 1.0 / (iteration + 1)  # Less noise as training progresses
    
    class_0 = np.random.randn(30, 512) * noise_level
    class_1 = np.random.randn(35, 512) * noise_level + 3
    class_2 = np.random.randn(35, 512) * noise_level - 3
    
    return np.vstack([class_0, class_1, class_2])

# Fit PCA on initial embeddings
initial_embeddings = simulate_model_embeddings(0)
scaler = StandardScaler()
initial_scaled = scaler.fit_transform(initial_embeddings)
pca = PCA(n_components=3)
pca.fit(initial_scaled)

print("Fitted PCA transformation")
print("Now we can reuse this transformation for all iterations...\n")

# Visualize progression
fig = plt.figure(figsize=(15, 4))
iterations = [0, 2, 5]

for idx, iteration in enumerate(iterations):
    # Get embeddings for this iteration
    emb = simulate_model_embeddings(iteration)
    
    # Transform using SAME PCA
    emb_scaled = scaler.transform(emb)
    emb_3d = pca.transform(emb_scaled)
    
    # Plot
    ax = fig.add_subplot(1, 3, idx + 1, projection='3d')
    for class_id in range(3):
        mask = labels == class_id
        ax.scatter(
            emb_3d[mask, 0],
            emb_3d[mask, 1],
            emb_3d[mask, 2],
            c=colors[class_id],
            s=30,
            alpha=0.6
        )
    ax.set_title(f'Iteration {iteration}')
    ax.set_xlabel('PC1')
    ax.set_ylabel('PC2')
    ax.set_zlabel('PC3')

plt.tight_layout()
plt.show()

print("Notice how classes cluster better as iterations progress!")

## Summary

**PCA Workflow:**
1. **Standardize** high-dimensional data (mean=0, std=1)
2. **Fit PCA** to find principal components
3. **Transform** to 3D coordinates
4. **Visualize** to understand clustering

**In our pipeline:**
- We fit PCA once on initial model embeddings
- We reuse the same transformation for all subsequent iterations
- This ensures consistent visualization as the model trains

**What to look for:**
- ✅ Classes forming distinct clusters = model is learning
- ❌ All mixed together = model needs more training/data
- 🔄 Clusters getting tighter over time = active learning is working!